# Лабораторная работа №3
## Выполнил студент группы БВТ2005 Иванов Никола

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
txt = input('Введите строку: ')
sub_txt = input('')

sens = input('чувствительность ? ')
if (sens == "нет"):
    txt = txt.lower()
    sub_txt = sub_txt.lower()

def prefix_function(text):
    n = len(text)
    pi = [0] * n  
    for i in range(1, n):
        j = pi[i-1]
        while j > 0 and text[j] != text[i]:
            j = pi[j-1]
        if text[i] == text[j]:
            j += 1
        pi[i] = j
    return pi


def kmp_search(text, sub_text, start_index=0):
    j = 0  
    pi = prefix_function(sub_text)  
    for i in range(start_index, len(text)):
        while j > 0 and text[i] != sub_text[j]:
            j = pi[j-1]
        if text[i] == sub_text[j]:
            j += 1
        if j >= len(sub_text):
            return i - j + 1
    return None

start_time = t.now()
print(' индекс начала: ', kmp_search(txt, sub_txt, 0))

end_time = t.now()
print('Duration: {}'.format(end_time - start_time))

Введите строку: Behaviour gave thoughts joy contrasted living judgment only morning unable endeavor doubtful. Determine collecting appear add narrow incommode companions answer living entire unknown. Wondered addition spot continued only defer time praise. Decisively feet visit excited raillery calling neglected tore begin outlived intention into resolving shade. Cordially object arise offered lady quit fulfilled returned savings against and shade mind message them. 
arise
чувствительность ? нет
 индекс начала:  367
Duration: 0:00:00


# Упрощенный алгоритм Бойера-Мура

In [3]:
txt = input('Введите строку: ')
sub_txt = input('поиск: ')

sens = input('чувствительность ?: ')
if (sens == "нет") :
    txt = txt.lower()
    sub_txt = sub_txt.lower()

start_time = t.now()

S = set()                # уникальные символы в образе
M = len(sub_txt)         # число символов в образе
d = {}                   # словарь смещений

for i in range(M - 2, -1, -1): 
    if sub_txt[i] not in S: 
        d[sub_txt[i]] = M - i - 1
        S.add(sub_txt[i])

if sub_txt[M - 1] not in S:  
    d[sub_txt[M - 1]] = M

d['*'] = M  

print(d)

N = len(txt)
if N >= M:
    i = M - 1  
    while (i < N):
        k = 0
        j = 0
        broke = False
        for j in range(M - 1, -1, -1):
            if txt[i - k] != sub_txt[j]:
        
                if j == M - 1:
                    off = d[txt[i]] if d.get(txt[i], False) else d['*']  
                    
                else:
                    off = d[sub_txt[j]]  
                    
                i += off  
                broke = True 
                break
            k += 1  
        if not broke:  
            print(f"Подстрока найдена по индексу: {i - k + 1}")
            break
    else:
        print(" не найдено")
else:
    print("Подстрока не найдена") 

end_time = t.now()
print('Duration: {}'.format(end_time - start_time))

Введите строку: Behaviour gave thoughts joy contrasted living judgment only morning unable endeavor doubtful. Determine collecting appear add narrow incommode companions answer living entire unknown. Wondered addition spot continued only defer time praise. Decisively feet visit excited raillery calling neglected tore begin outlived intention into resolving shade. Cordially object arise offered lady quit fulfilled returned savings against and shade mind message them. 
поиск: arise
чувствительность ?: нет
{'s': 1, 'i': 2, 'r': 3, 'a': 4, 'e': 5, '*': 5}
Подстрока найдена по индексу: 367
Duration: 0:00:00.000998


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [4]:
from queue import PriorityQueue

N = 4

def moves(position):
    blank = position.index(0) 
    i, j = divmod(blank, N) 
    offsets = []
    if i > 0: offsets.append(-N)     #  вниз
    if i < N - 1: offsets.append(N)  #  вверх
    if j > 0: offsets.append(-1)     #  вправо
    if j < N - 1: offsets.append(1)  #  влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

def parity(permutation):
    seen, cycles = set(), 0 
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:
    
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance
    
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

if parity(start) == 0:
    print('Задачу решить нельзя')
else:
    start = tuple(start) 
    p = Position(start, 0) 
    print(p) 
    print()

    candidates = PriorityQueue() # конструктор для очереди с приоритетом
    candidates.put(p)

    # посещ. поз.
    visited = set([p])

    came_from = {p.position: None} # словарь 
    
    while p.position != SOLVED:
        # Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # Кладем в очередь все соседние позиции. Повторяем не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    path = []
    prev = p
    while p.position != start:
        # Запоминаем откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]
